## ruGPT3 Finetuning

Попробуем натренировать сеть ruGPT3 на фразах переписки из Телеграм. Для обучения необходим датасет вот такого формата:
```
<s>Норм, но сейчас физика, мы умираем</s>
<s>Мама должна будет забрать</s>
<s>Да, скорее всего</s>
<s>Жестоко)</s>
<s>​ Возможно именно так выглядела эволюция алфавита. Класс, да. Получается, что буква А произошла от иероглифа, изображающего голову животного с рогами, а буква С поначалу была похожа на палку охотника или бумеранг. Некоторые буквы вообще потерялись, а такие как U, V и W, возникли из одного символа.

Это 3800 лет истории. Путь от египетских иероглифов через финикийский, древнегреческий и латинский алфавит – до нынешнего языка.</s>
<s>Готовься</s>
```

Для подготовки такого датасета из своей переписки используйте код из `ConvConv.ipynb`. Поместите датасет в файл `train.txt`

Посмотрим на то, как выглядит датасет:

In [1]:
!head train.txt

<s>FF</s>
<s>da</s>
<s>ну вот(</s>
<s>спасибо</s>
<s>да</s>
<s>Па, а у тебя камера с собой?</s>
<s>Или в номере?</s>
<s>Что стоит делать?</s>
<s>Ок</s>
<s>Привет</s>


Установим необходимые библиотеки. Важно соблюсти правильную версию датасета `transformers`, поскольку она связана с файлом для обучения `run_clm.py`, который мы адаптировали для Datasphere. 

In [2]:
%pip install -U transformers==4.30.2 accelerate evaluate datasets==4.0

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.3/564.3 kB 7.6 MB/s  0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2021.11.1
    Uninstalling fsspec-2021.11.1:
      Successfully uninstalled fsspec-2021.11.1
  Attempting uninstall: huggingface-hub━━━━━━━━━ 0/5 [fsspec]
    Found existing installation: huggingface-hub 0.17.3/5 [fsspec]
    Uninstalling huggingface-hub-0.17.3:━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [huggingface-hub]
      Successfully uninstalled huggingface-hub-0.17.3━━━━━━━━━━━━━ 1/5 [huggingface-hub]
  Attempting uninstall: datasets90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/5 [huggingface-hub]
    Found existing installation: datasets 2.11.0━━━━━━━━━━━━━━ 1/5 [huggingface-hub]
    Uninstalling datasets-2.11.0:╺━━━━━━━━━━━━━━━━━━━━━━━ 2/5 [datasets]ub]
      Successfully uninstalled datasets-2.11.0━━━━━━━━━━━━━━━━━━━━ 2/5 [datasets]
  Attempting uninstall: evaluatem╺━━━━━━━━

Мы взяли скрипт `run_clm.py` из библиотеки `transformers` и слегка адаптировали его для работы в Yandex Datasphere. Импортируем из него нужные объекты:

In [2]:
from run_clm import TrainingArguments, ModelArguments, DataTrainingArguments, main

2025-10-01 21:31:45.000587: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


И теперь запускаем обучение, установив основные параметры:
* Файл с обучающим датасетом
* Имя базовой модели, которую будем дообучать
* Число эпох обучения
* Размер текстового блока и размер обучающего батча (это параметр подбирается исходя из доступной видеопамяти GPU)

In [3]:
main(
    ModelArguments(
        model_name_or_path="sberbank-ai/rugpt3small_based_on_gpt2"),
    DataTrainingArguments(
        train_file='train.txt',
        dataset_config_name='plain_text',
        block_size=2048),
    TrainingArguments(
        output_dir="models",
        overwrite_output_dir=True,
        num_train_epochs=10,
        per_device_train_batch_size=1,
        do_train=True,
        log_level='error')
)


/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 85%|████████▍ | 500/590 [06:48<01:14,  1.20it/s]

{'loss': 2.5722, 'learning_rate': 7.627118644067798e-06, 'epoch': 8.47}


100%|██████████| 590/590 [08:46<00:00,  1.12it/s]


{'train_runtime': 526.3909, 'train_samples_per_second': 1.121, 'train_steps_per_second': 1.121, 'train_loss': 2.494899710962328, 'epoch': 10.0}
***** train metrics *****
  epoch                    =       10.0
  train_loss               =     2.4949
  train_runtime            = 0:08:46.39
  train_samples            =         59
  train_samples_per_second =      1.121
  train_steps_per_second   =      1.121


ConnectionError: (ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')), '(Request ID: 6dd733d4-fedd-451b-920d-d38c58d1533f)')

Теперь в диретории `models` получилась обученная модель. Можем загрузить её и попробовать генерацию текста начиная со стартового токена:

In [4]:
from transformers import pipeline, AutoModelForCausalLM,AutoTokenizer
import torch

model_name = 'models'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipe = pipeline(model=model,tokenizer=tokenizer,task="text-generation",device="cuda:0")

In [36]:
result = pipe("<s>ого, ",do_sample=True,max_length=1500)[0]['generated_text'].replace('\\n','\n')
result

/home/jupyter/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'<s>ого,  а ты умеешь ходить? :('

## Мораль

Даже сравнительно на небольшом датасете можно обучить небольшую модель имитировать стиль!